In [178]:
import pandas as pd
import numpy as np
import random
import sklearn

In [179]:
companies = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/companies.csv").dropna()

In [180]:
companies = companies.convert_dtypes(convert_string=True)

In [181]:
companies_array = np.array(companies).flatten()

In [182]:
df_train = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/train_turkish.csv").convert_dtypes(convert_string=True).dropna()
splits = {'train': 'train.csv', 'test': 'test.csv'}
df_train = pd.read_csv("hf://datasets/winvoker/turkish-sentiment-analysis-dataset/" + splits["test"])

In [183]:
length = 0
for text in df_train["text"]:
    length += len(text)
print(length/len(df_train["text"]))

all_lengths = []

for text in df_train["text"]:
    all_lengths.append(len(text))
print(np.median(all_lengths))

140.11768203159215
82.0


In [184]:
def cut_text(text:str):
    splitted = str(text).split()
    splitted = splitted[:15]
    return " ".join(splitted)

In [185]:
df_train["text"] = df_train["text"].map(cut_text)

In [186]:
df_train_texts = df_train["text"].to_list()

In [187]:
labels = {"Notr":1,"Positive":3,"Negative":2}

In [188]:
sentiment = df_train["label"].map(str).to_numpy()

In [189]:
sentiment[0]

'Positive'

In [190]:
sentiment = np.apply_along_axis(lambda x : labels[x[0]],axis=1,arr=sentiment.reshape((-1,1)))

In [191]:
notr_bow = pd.read_csv("/home/musasina/projects/teknofest/msnet/bows/notr_bow_not_cleared_test.csv")
positive_bow = pd.read_csv("/home/musasina/projects/teknofest/msnet/bows/positive_bow_not_cleared_test.csv")
negative_bow = pd.read_csv("/home/musasina/projects/teknofest/msnet/bows/negative_bow_not_cleared_test.csv")

In [192]:
notr_bow_set = set(notr_bow["words"].map(str).to_list())
positive_bow_set = set(positive_bow["words"].map(str).to_list())
negative_bow_set = set(negative_bow["words"].map(str).to_list())

In [193]:
wrong_bow_set = notr_bow_set.intersection(positive_bow_set,negative_bow)
wrong_bow_set_pos_notr = notr_bow_set.intersection(positive_bow_set)
wrong_bow_set_pos_neg = negative_bow_set.intersection(positive_bow_set)
wrong_bow_set_neg_notr = negative_bow_set.intersection(notr_bow_set)

In [194]:
print(len(wrong_bow_set_pos_notr))
print(len(wrong_bow_set_pos_neg))
print(len(wrong_bow_set_neg_notr))

11444
20136
8921


In [195]:
random.seed(42)

In [196]:
notr_bow_set -= wrong_bow_set
positive_bow_set -= wrong_bow_set
negative_bow_set -= wrong_bow_set
#----------------------------------
positive_bow_set -= set(random.sample(list(wrong_bow_set_pos_neg),len(wrong_bow_set_pos_neg)//2))
negative_bow_set -= set(random.sample(list(wrong_bow_set_neg_notr),len(wrong_bow_set_neg_notr)//2))
positive_bow_set -= set(random.sample(list(wrong_bow_set_pos_notr),len(wrong_bow_set_pos_notr)//2))
#----------------------------------

In [197]:
print(len(notr_bow_set))
print(len(positive_bow_set))
print(len(negative_bow_set))

23455
30451
21362


In [198]:
texts_with_companies = []

for text,label in zip(df_train_texts,sentiment):
    for _ in range(1):
        splitted_text = text.split()[:15]
        if len(splitted_text) == 15:
            index = np.random.randint(0,len(splitted_text))
            company = np.random.choice(companies_array)
            labels = [0]*15
            for i,word in enumerate(splitted_text):
                if word in notr_bow_set:
                    labels[i]=1
                if word in negative_bow_set:
                    labels[i]=2
                if word in positive_bow_set:
                    labels[i]=3
            labels.insert(index,label)
            splitted_text.insert(index,company)
            texts_with_companies.append({
                "text":" ".join(splitted_text),
                "sentiment":label,
                "labels":labels
            })

In [199]:
import sklearn.utils


df_final = sklearn.utils.shuffle(pd.DataFrame(texts_with_companies),random_state=42)
df_final.to_csv("/home/musasina/projects/teknofest/msnet/datasets/test_final.csv",index=False)